# <b style='color:green;'> Treinamento</b>
* Este Jupyter tem como realizar função realizar o treinamento do modelo.

---

Bibliotecas/Módulos

<details>    
<summary>
    <font size="3" color="green"><b>Install</b></font>
</summary>
<p>
<ul>
    <li> !pip install sklearn </li>
    <li> !pip install pandas </li>
    <li> !pip install scikit-multilearn </li>
</ul>
</p>

In [1]:
%%time
%%capture
%run 01Analise.ipynb

CPU times: user 2.95 s, sys: 83.7 ms, total: 3.03 s
Wall time: 3.03 s


In [2]:
import numpy as np
import torch
import wandb

from simpletransformers.classification import MultiLabelClassificationModel

---

Dados que serão utilizados

In [3]:
features.sample()

,text,label
2178,utilizando package code websocket fazer transm...,"(1, 0, 0, 0)"


Separando os dados para `treino` e `validação`.

In [4]:
train_df, eval_df = train_test_split(features, test_size = 0.2, random_state = 123)

#### Cuda utilizável?

In [5]:
cuda_available = torch.cuda.is_available()
cuda_available

True

---

## Sweep

Vamos utilizar o Wandb e o Sweep encontrar os melhores parâmetros para o nosso modelo BERT final.

Primeiramente será necessário realizar login na plataforma.

```bash
!wandb login
```

Posteriormente, setar as configurações para este projeto.

<a href="https://wandb.ai/saraselisn/bert-multiclass"> WandbPag </a>

> Configurações do Sweep

Esses serão os parâmetros estudados pelo wandb e sweep.

In [6]:
sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [1, 2, 3, 5]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
        "train_batch_size": {"values": [4, 8, 16, 32]},
        "eval_batch_size": {"values": [4, 8, 16, 32]}
    },
}

In [7]:
sweep_id = wandb.sweep(sweep_config)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


Create sweep with ID: m5pfnkqd
Sweep URL: https://wandb.ai/saraselisn/uncategorized/sweeps/m5pfnkqd


In [8]:
!wandb login cb98f35984005bb9708fdaf391cb19de812751ea

wandb: Appending key for api.wandb.ai to your netrc file: /home/stefanini/.netrc


In [9]:
def train():
    # inicializando wandb/sweep
    wandb.init(project="bertmulti", entity="teamname")

    # Instanciando o modelo
    model = ClassificationModel(
        model_type="roberta",
        model_name="roberta-base",
        num_labels=4,
        use_cuda=True,
        args=args,
        sweep_config=wandb.config,
    )

    # treinamento/validacao
    model.train_model(train_df, eval_df=eval_df)

    # sync wandb
    wandb.join()

In [10]:
treino = wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 1qdr1s02 with config:
wandb: 	eval_batch_size: 16
wandb: 	learning_rate: 7.277075811409873e-05
wandb: 	num_train_epochs: 3
wandb: 	train_batch_size: 8
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: saraselisn (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Run 1qdr1s02 errored: NameError("name 'ClassificationModel' is not defined")
wandb: ERROR Run 1qdr1s02 errored: NameError("name 'ClassificationModel' is not defined")
wandb: Agent Starting Run: ofkos49l with config:
wandb: 	eval_batch_size: 4
wandb: 	learning_rate: 0.0001747507792501886
wandb: 	num_train_epochs: 2
wandb: 	train_batch_size: 32
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Run ofkos49l errored: NameError("name 'ClassificationModel' is not defined")
wandb: ERROR Run ofkos49l errored: NameError("name 'ClassificationModel' is not defined")
wandb: Agent Starting Run: nblmf311 with config:
wandb: 	eval_batch_size: 4
wandb: 	learning_rate: 0.00039563696681335533
wandb: 	num_train_epochs: 5
wandb: 	train_batch_size: 4
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Run nblmf311 errored: NameError("name 'ClassificationModel' is not defined")
wandb: ERROR Run nblmf311 errored: NameError("name 'ClassificationModel' is not defined")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


Nesta análise, pudemos observar que os parâmetros selecionados não inteferiram no resultado final, apenas em questão de performance no treinamento do modelo.
Neste caso, pouco interferem os argumentos utilizados.

In [11]:
args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'use_multiprocessing': False,
    "num_train_epochs": 1,
    "train_batch_size": 16,
    "eval_batch_size": 16,
}

----

## Treinamentos

### Treinamento Roberta

In [13]:
model = MultiLabelClassificationModel("roberta", "roberta-base", num_labels=4, use_cuda=True, args=args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [14]:
model.train_model(train_df, eval_df=eval_df)

/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:377: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/home/stefanini/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/home/stefanini/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(271,
 {'global_step': [271],
  'LRAP': [0.7874306839186698],
  'train_loss': [0.318287193775177],
  'eval_loss': [0.4253999923958498]})

In [15]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [16]:
result

{'LRAP': 0.7874306839186698, 'eval_loss': 0.4253999923958498}

Testando modelo

In [17]:
text = features['text'][4]

In [18]:
print(BLUE + text)

exibir dados retornados firestore diretiva angular ngmodel gostaria pegar dados retornado firestore exibir diretiva angular ngmodel exibindo tela edicao pessoa model ts code pessoa service ts code pessoa view ts code pessoa edit component html code


In [19]:
predictions, probs = model.predict([text])

#### tags
<b> node.js	jquery	html	angular

In [20]:
predictions

[[0, 0, 0, 1]]

In [21]:
probs

array([[0.17712402, 0.0960083 , 0.14868164, 0.88232422]])

### Treinamento BERT

In [26]:
model = MultiLabelClassificationModel("bert", "bert-base-cased", num_labels=4, use_cuda=True, args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not in

In [27]:
model.train_model(train_df, eval_df=eval_df)

/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:377: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/home/stefanini/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/home/stefanini/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(271,
 {'global_step': [271],
  'LRAP': [0.827479975354283],
  'train_loss': [0.38590025901794434],
  'eval_loss': [0.38024457390694055]})

In [35]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/home/stefanini/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [36]:
result

{'LRAP': 0.827479975354283, 'eval_loss': 0.38024457390694055}

Testando modelo

In [37]:
text = features['text'][4]

In [38]:
print(BLUE + text)

exibir dados retornados firestore diretiva angular ngmodel gostaria pegar dados retornado firestore exibir diretiva angular ngmodel exibindo tela edicao pessoa model ts code pessoa service ts code pessoa view ts code pessoa edit component html code


In [39]:
predictions, probs = model.predict([text])

#### tags
<b> node.js	jquery	html	angular

In [40]:
predictions

[[0, 0, 0, 1]]

In [41]:
probs

array([[0.13916016, 0.07849121, 0.16662598, 0.92529297]])

A métrica <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.label_ranking_average_precision_score.html"> LRAP </a> calcula a precisão média para cada uma das labels acertadas dentro do conjunto `(1, 0, 0, 0)`. Diferente de outras métricas que punem todo o conjunto caso alguma das labels esteja errada.

Esta métrica é uma boa escolha para resolver este problema, pois em uma classificação de perguntas em um fórum de dúvidas na internet, caso o modelo venha a errar apenas uma das marcações não gera tantos problemas como aconteceria em outras ocasiões.

---

Neste caso, o modelo BERT tobteve números melhores e este será o modelo utilizado.

In [42]:
model.save_model(PATH_GERAL + "/Pesos/bert_multicass", model=model.model)